In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


sunspots = sm.datasets.sunspots.load_pandas().data


sunspots['YEAR'] = pd.to_datetime(sunspots['YEAR'].astype(str), format='%Y')
sunspots = sunspots.set_index('YEAR')


plt.figure(figsize=(12, 6))
plt.plot(sunspots.index, sunspots['SUNACTIVITY'], label='Sunspot Activity')
plt.xlabel('Year')
plt.ylabel('Sunspot Activity')
plt.title('Sunspot Activity Over Time')
plt.legend()
plt.show()


decomposition = sm.tsa.seasonal_decompose(sunspots['SUNACTIVITY'], model='additive')
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid


plt.figure(figsize=(12, 6))
plt.subplot(411)
plt.plot(sunspots.index, sunspots['SUNACTIVITY'], label='Original')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(sunspots.index, trend, label='Trend')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(sunspots.index, seasonal, label='Seasonal')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(sunspots.index, residual, label='Residual')
plt.legend(loc='upper left')
plt.tight_layout()


order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)
sarima_model = sm.tsa.SARIMAX(sunspots['SUNACTIVITY'], order=order, seasonal_order=seasonal_order)
sarima_result = sarima_model.fit()


print(sarima_result.summary())

sarima_result.plot_diagnostics()
plt.show()


forecast_periods = 36  
forecast = sarima_result.get_forecast(steps=forecast_periods)


forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()


plt.figure(figsize=(12, 6))
plt.plot(sunspots.index, sunspots['SUNACTIVITY'], label='Observed')
plt.plot(forecast_mean.index, forecast_mean, color='red', label='Forecast')
plt.fill_between(forecast_ci.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink')
plt.xlabel('Year')
plt.ylabel('Sunspot Activity')
plt.title('Sunspot Activity Forecast with SARIMA')
plt.legend()
plt.show()
